# Model / Class driven events and quest management

Events are happening all the time. Several events happen before the Character quest begins. 

In [25]:
import pandas as pd
import numpy as np
import os

import os
import sys
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()

#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys,pickle
sys.path.append('../')


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [26]:
from game.models import events

In [27]:
def get_possible_events():
    return [str(i[0]) for i in events.objects.values_list('key')]

def get_event(k):
    m = events.objects.get(pk=k)
    return m

possible_events = get_possible_events()
possible_events

['bandits', 'marrage', 'raiders', 'temple']

as a convention I'm adding `e` as an individual event as the result of the DB query. 

In [111]:
e = get_event('temple')
e

<events: events object (temple)>

I'm creating an events class to manage individual events. I origionally thought of doing this within the `game.models` but I wanted to have something that reacts to individual events and manages all of the effects that they have on the world. 

In [135]:
class Event:
    def __init__(self,e):
        self.key = e.key
        self.n_subjects = e.n_subjects
        self.n_objects = e.n_objects
        self.event = e.event
        self.effect_var = e.effect_var
        self.effect = e.effect
        self.a_message = e.a_message
        self.o_message = e.o_message
        self.agent_type= e.agent_type
        self.a = None
        self.o = None
        self.t = None
    
    def __str__(self):
        return self.event.format(a=self.a,o=self.o,t=self.t)
    
    def get_event_propperties(self):
        ep = {
            "key":self.key,
            "n_subjects":self.n_subjects,
            "n_objects":self.n_objects,
            "event":self.event,
            "effect_var":self.effect_var,
            "effect":self.effect,
            "a_message":self.a_message,
            "o_message":self.o_message,
            "agent_type":self.agent_type
        }
        if self.a:
            ep['ep'] = self.a
        return ep
    
    def random_assign_agent(self,world):
        #1) figure out who did what, and to whom
        if self.agent_type == "nation":
            if self.n_subjects > 0:
                self.a = np.random.choice(world.nations,self.n_subjects,replace=False)
            if self.n_objects > 0:
                self.o = np.random.choice(world.nations,self.n_objects,replace=False)
        if '{t}' in self.event:
            if self.o:
                terrain_in_nation = world.df_features.loc[world.df_features['nation'].isin([e.name for e in event.o]),'terrain'].unique()
                terrain_in_nation = [t for t in terrain_in_nation if t!='town']
                self.t = np.random.choice(terrain_in_nation,replace=False)
            else:
                terrain_in_nation = np.unique(world.df_features['terrain'])
                self.t = np.random.choice(terrain_in_nation,replace=False)
        return self.event
    
    def assign_effects(self,world):
        pass
        
event = Event(e)
print(event)
event.get_event_propperties()

a great temple was founded in the city of None in None


{'key': 'temple',
 'n_subjects': 1,
 'n_objects': 0,
 'event': 'a great temple was founded in the city of {t} in {a}',
 'effect_var': 'buildings',
 'effect': Decimal('0.00'),
 'a_message': 'did you hear about the great temple in {a}',
 'o_message': 'hello',
 'agent_type': 'nation'}

In [136]:
event.random_assign_agent(world)
print(event)

a great temple was founded in the city of forest in [Nation of Landsplace]


## Test: All `Possible events` can be created
And have no `None` types in the string

In [138]:
for key in possible_events:
    e = get_event(key)
    event = Event(e)
    event.random_assign_agent(world)
    print(event)

bandits have set up camp in the desert of [Nation of Brinehaven]
a noble from [Nation of Fairforth] was married to a noble in [Nation of Snakerun]
raiders from [Nation of Landsfellow], plundered [Nation of Earthbrook Nation of Snakerun Nation of Brinehaven]
a great temple was founded in the city of town in [Nation of Brinecombe]
